In [1]:
import time, os, sys
import tweepy
import pandas as pd
from tqdm import tqdm
from datetime import datetime,timezone
import pytz

In [2]:
sys.path.append("../")  # ローカル
import env.config  # ローカルのキーを取りに行く

CONSUMER_KEY = os.environ.get("CONSUMER_KEY")  # Consumer Key
CONSUMER_SECRET = os.environ.get("CONSUMER_SECRET")  # Consumer Secret
ACCESS_TOKEN = os.environ.get("ACCESS_TOKEN_KEY")  # Access Token
ACCESS_TOKEN_SECRET = os.environ.get("ACCESS_TOKEN_SECRET")  # Accesss Token Secert

In [3]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth ,wait_on_rate_limit = False)

In [4]:
#フォロワー一覧を取得
search_id = "openmeti"

In [5]:
# if文にてRTとリプライを除外
[
    tweet.text
    for tweet in tweepy.Cursor(api.user_timeline, id=search_id).items(20)
    if (list(tweet.text)[:2] != ["R", "T"]) & (list(tweet.text)[0] != "@")
]

Unexpected parameter: id


['9/17スタート！「Code for Japan Summit 2022」に経済産業省DX室が今年も登壇します。経産省のDX推進体制、DX事例、これからのチャレンジのほか、採用情報もご紹介。ぜひご覧ください！… https://t.co/vkcVWRPw4y',
 '経済産業省の「DXロゴ」がオープンデータ化されました！ロゴデータ、フォントのセットで、オリジナルのDXロゴの作成が可能です。\n当省のDXの取り組みに賛同いただける行政機関の皆様、行政DXの推進のために、ぜひ奮ってご活用ください！！\nhttps://t.co/pez8KsCuXc',
 '経済産業省のDXの取組事例について紹介します！\n▼詳しくは、こちらのnoteから！\nhttps://t.co/2urDERSc0F\n\n▼経済産業省のDXを推進するIT人材を募集しています！\n#DX #デジタル化 #デジタル人材… https://t.co/7BvHg8eo3i',
 'DXに必要な行政職員のスキルとは？\n▼詳しくは、こちらのnoteから！\nhttps://t.co/9jGi6QTmc0\n\n▼経済産業省のDXを推進するIT人材を募集しています。\n#DX #デジタル化 #デジタル人材 #IT人材 https://t.co/p2twofYqyq',
 '経済産業省DXオフィスでは、以下のポジションを募集してます。\n①ITアドバイザリー\nhttps://t.co/7c5xfuaQW9\n②ローコードエンジニア兼ITアドバイザリー\nhttps://t.co/aAnB5k0NXu\n③デー… https://t.co/3Mrz8nqJ9u',
 '▼ローコード開発ツールは行政デジタル化の救世主となるか！詳しくはこちらのnoteから！\nhttps://t.co/6InOU9RApA\n\n▼経済産業省のIT人材（ローコードエンジニア）転職はこちら！… https://t.co/6f64egd7fk',
 '経済産業省にデータサイエンティストがJOINし、どう変わるのか！\n▼詳しくはこちらのnoteから！\nhttps://t.co/ZpAYQZPXfr\n▼経済産業省のIT人材（データエンジニア）転職はこちら！\nhttps://t.co/sP0kxEeSFC',
 'また、民間I

In [6]:
# 検索条件の設定
search_word = "DX 経産省 min_faves:2"
# 何件のツイートを取得するか
item_number = 150

# 検索条件を元にツイートを抽出
tweets = tweepy.Cursor(
    api.search_tweets,
    q=search_word,
    tweet_mode="extended",
    result_type="mixed",
    lang="ja",
).items(item_number)

# 関数:　UTCをJSTに変換する
def change_time_JST(u_time):
    # イギリスのtimezoneを設定するために再定義する
    utc_time = datetime(
        u_time.year,
        u_time.month,
        u_time.day,
        u_time.hour,
        u_time.minute,
        u_time.second,
        tzinfo=timezone.utc,
    )
    # タイムゾーンを日本時刻に変換
    jst_time = utc_time.astimezone(pytz.timezone("Asia/Tokyo"))
    # 文字列で返す
    str_time = jst_time.strftime("%Y-%m-%d_%H:%M:%S")
    return str_time


tw_data = []
for tweet in tweets:

    tweet_time = change_time_JST(tweet.created_at)
    create_account_time = change_time_JST(tweet.user.created_at)
    
    tw_data.append(
        [
            tweet.id,
            tweet_time,
            tweet.full_text,
            tweet.favorite_count,
            tweet.retweet_count,
            tweet.user.id,
            tweet.user.screen_name,
            tweet.user.name,
            tweet.user.description,
            tweet.user.friends_count,
            tweet.user.followers_count,
            create_account_time,
            tweet.user.following,
            tweet.user.profile_image_url,
            tweet.user.profile_background_image_url,
            tweet.user.url,
        ]
    )


labels = [
    "ツイートID",
    "ツイート時刻",
    "ツイート本文",
    "いいね数",
    "リツイート数",
    "ID",
    "ユーザー名",
    "アカウント名",
    "自己紹介文",
    "フォロー数",
    "フォロワー数",
    "アカウント作成日時",
    "自分のフォロー状況",
    "アイコン画像URL",
    "ヘッダー画像URL",
    "WEBサイト",
]

# tw_dataのリストをpandasのDataFrameに変換
pd.DataFrame(tw_data, columns=labels)

,ツイートID,ツイート時刻,ツイート本文,いいね数,リツイート数,ID,ユーザー名,アカウント名,自己紹介文,フォロー数,フォロワー数,アカウント作成日時,自分のフォロー状況,アイコン画像URL,ヘッダー画像URL,WEBサイト
0,1569903331669344258,2022-09-14_13:18:16,筑波銀行、経産省のDX認定事業者に 北関東の地銀で初: 日本経済新聞 https://t.c...,8,1,104745960,yoshitaka_kitao,北尾吉孝,1974年慶応義塾大学経済学部卒業後野村證券入社。1978年英国ケンブリッジ大学経済学部卒業...,630,122583,2010-01-14_16:46:44,False,http://pbs.twimg.com/profile_images/5041537858...,http://abs.twimg.com/images/themes/theme1/bg.png,https://t.co/qAlwWo9F4d
1,1569867270738038785,2022-09-14_10:54:58,[New!]\n「ゲーミフィケーション」を活用したDX人材育成に関する調査・検討会を経産省が...,2,2,3241066586,keieipro,経営プロ,ProFuture株式会社が提供する経営層向けポータルサイト『経営プロ』公式アカウントです。...,172,394,2015-06-10_15:11:40,False,http://pbs.twimg.com/profile_images/8770118724...,http://abs.twimg.com/images/themes/theme1/bg.png,http://t.co/QAPyMu8L1N
2,1569845400047337474,2022-09-14_09:28:04,国のお金でフィリピンを支援する契約が締結されました。\nDX、デジタルトランスインフォーメー...,2,5,1415991922234060801,MafuTugumi,mafu_tugumi🍀子どもの後遺症と軽症致死を減らしたい,不具合たびたび有〼🙇‍♀️新垢時は固定ツイ一番下にリプ予定。コメントや返信等の読取不可や非表...,3326,923,2021-07-16_20:10:01,False,http://pbs.twimg.com/profile_images/1519609939...,None,None
3,1569806750567780353,2022-09-14_06:54:29,【有益】\nそういえば、経産省のマナビDX、無料で学べるからおすすめですよ。 https:/...,98,15,1228690595960385536,totonoe_san,ととのえ｜令和の生き方･働き方,複業しながら自分らしく生きることを追求してる管理職サラリーマン。令和の生き方や働き方について...,229,19305,2020-02-15_23:40:49,False,http://pbs.twimg.com/profile_images/1543831506...,None,https://t.co/qBXgsI7ryP
4,1569133291856433152,2022-09-12_10:18:24,「IT人材がいない」は、本当か？\n\n日本企業に警鐘を鳴らした「2025年の崖」まで、あと...,6,1,1131098394326233088,NewsPicksBDT,NewsPicks Brand Design,ビジネスパーソンに役立つ情報を、企業スポンサードで無料でお届け⚡️読み物やインフォグラフィッ...,89,3881,2019-05-22_16:24:06,False,http://pbs.twimg.com/profile_images/1131098562...,None,https://t.co/DVB64ZWZfC
5,1568182830139846657,2022-09-09_19:21:36,デジタルとかDXとか気になる人はまじで参考になるから使ってみてほしい。最近暇さえあればこれ見...,16,0,1149204047322177536,shukatsugamer,就活ゲーマー,Noゼミ/Noサークル/Noバイト/No留学/No TOEIC/ガクチカはPCゲームのみ🎮/...,736,3738,2019-07-11_15:29:30,False,http://pbs.twimg.com/profile_images/1448406554...,None,https://t.co/lL1kW7skWr
6,1567734059576737792,2022-09-08_13:38:21,9/17スタート！「Code for Japan Summit 2022」に経済産業省DX室...,8,3,92026643,openmeti,経済産業省DXオフィス,経済産業省におけるデジタル・トランスフォーメーション推進twitterです。 twitter...,135,102588,2009-11-23_23:10:45,False,http://pbs.twimg.com/profile_images/591847786/...,http://abs.twimg.com/images/themes/theme13/bg.gif,https://t.co/3lku9j0Bsc
7,1567475821534941184,2022-09-07_20:32:12,経産省が出しているDX情報は企業向けのもので、ITに詳しくない人には少し想像しにくい内容です...,2,0,1432946692710547457,miraikaigi_dbs,DXみらいラジオ📻,株式会社デジタルビジネスシェアリングが提供する「聞いてるだけでDXがわかるラジオ」 システム...,1047,362,2021-09-01_15:02:26,False,http://pbs.twimg.com/profile_images/1488686558...,None,https://t.co/NW4qX73Mv7
8,1567442259024683009,2022-09-07_18:18:50,@kaji_fish_dx 僕の所感ですが、水産のベンチャーってあまり聞かない分、相対的に評...,2,0,1438992993399160837,Zephyr_dawn,秋野 遊一@水産業従事者,魚の増養殖研究に取り組み、現在は陸上養殖に従事。RASの立ち上げを経験済み。何かお手伝いでき...,461,257,2021-09-18_07:29:47,False,http://pbs.twimg.com/profile_images/1438994795...,None,None
9,1567426131640598530,2022-09-07_17:14:45,キリンホールディングスや経産省の事例もありましたが、中小のソフトウェア開発会社だけではなく、...,4,1,1561530488791400448,NulabGrowth,個人アクティビスト@Nulab株主,個人で割安な株式を取得し経営に関する意見を発信していきます。現在はNulab株に投資をしてお...,208,41,2022-08-22_10:47:48,False,http://pbs.twimg.com/profile_images/1561534357...,None,None


In [7]:
followers_ids = tweepy.Cursor(api.get_follower_ids, id='openmeti', cursor=-1).items()

- 普通にループ回すと認証エラーになる
- 一定周期で認証し直す戦略 https://qiita.com/makimaki913/items/c081a8e85c004bd1bc7d
- 最初考えたコード

```{python}
follower_list = []

for follower_id in tqdm(followers_ids):
    user = api.get_user(user_id=follower_id)
    _user_info = [
        user.id_str,
        user.screen_name,
        user.name,
        user.created_at,
        user.location,
        user.description,
        user.verified,
        user.followers_count,
        user.friends_count,
        user.favourites_count,
        user.statuses_count,
    ]
    follower_list.append(_user_info)
```

In [ ]:
follower_list = []
cursor = -1
while cursor != 0:
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    api = tweepy.API(auth, wait_on_rate_limit=False)
    itr = tweepy.Cursor(api.get_follower_ids, id="openmeti", cursor=cursor).pages()
    try:
        for follower_id in itr.next():
            try:
                user = api.get_user(user_id=follower_id)
                _user_info = [
                    user.id_str,
                    user.screen_name,
                    user.name,
                    user.created_at,
                    user.location,
                    user.description,
                    user.verified,
                    user.followers_count,
                    user.friends_count,
                    user.favourites_count,
                    user.statuses_count,
                ]
                follower_list.append(_user_info)
            except :
                print("limit error")
    except ConnectionError as e:
        print(e.reason)
    cursor = itr.next_cursor

In [ ]:
# カラム の意味
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user

metidx_follower_df = pd.DataFrame(
    follower_list,
    columns=[
        "id_str",
        "screen_name",
        "name",
        "created_at",
        "location",
        "description",
        "verified",
        "followers_count",
        "friends_count",
        "favourites_count",
        "statuses_count",
    ],
)
metidx_follower_df

In [ ]:
metidx_follower_df.describe()